In [21]:
import re
import sys

In [22]:
class Node:
    def __init__(self, value, children=None):
        self.value = value
        self.children = children or []

    def to_dict(self):
        if not self.children:
            return {"type": "NumericLiteral", "value": self.value}
        elif len(self.children) == 1:
            return {"type": "UnaryExpression", "op": self.value, "argument": self.children[0].to_dict()}
        elif len(self.children) == 2:
            return {"type": "BinaryExpression", "op": self.value, "left": self.children[0].to_dict(), "right": self.children[1].to_dict()}

def tokenization(input_str):
    return input_str.split()

def is_parenthesis(ch):
    return ch == '(' or ch == ')'

def is_operator(ch):
    return ch in ['+', '-', '*', '/']

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

In [23]:
def parse_expr(tokens):
    # expr -> - expr
    if tokens[0] == '-' and is_operator(tokens[0]):
        tokens.pop(0)  # Consume the -
        if tokens[0] == '-':
            return None
        return Node('-', [parse_expr(tokens)])

    # expr -> ( expr )
    if tokens[0] == '(':
        tokens.pop(0)  # Consume the (
        inner_expr = parse_expr(tokens)
        if not inner_expr or not tokens or tokens[0] != ')':
            return None  # Missing )
        tokens.pop(0)  # Consume the )
        if tokens and is_operator(tokens[0]):
            # expr -> expr op expr
            operator = tokens.pop(0)
            return Node(operator, [inner_expr, parse_expr(tokens)])
        return inner_expr

    # expr -> id
    if is_number(tokens[0]):
        if tokens[0] == '-' and is_operator(tokens[0]):
            tokens.pop(0)  # Consume the -
            if tokens[0] == '-':
                return None
            return Node('-', [parse_expr(tokens)])

        value = tokens.pop(0)  # Consume the id
        if tokens and is_operator(tokens[0]):
            # expr -> expr op expr
            operator = tokens.pop(0)
            return Node(operator, [Node(value), parse_expr(tokens)])
        else:
            return Node(value)

    return None  # Token is not an identifier or a valid expression

def syntax_analyzer(tokens):
    return parse_expr(tokens) and not tokens

def print_ast_dict(parsed_dict, indent=0):
    for key, value in parsed_dict.items():
        if isinstance(value, dict):
            print(" " * indent + f"{key}:")
            print_ast_dict(value, indent + 2)
        else:
            print(" " * indent + f"{key}: {value}")

In [24]:

def main():
    while True:
        value = input("Enter a String (Type 'exit' to terminate): ")

        if value.lower() == "exit":
            break

        tokens = tokenization(value)
        print("------------------------")
        print("Lexical Analysis : ")
        print("------------------------")

        for token in tokens:
            print(token, end=' ')
            if is_number(token):
                print("Is a Value")
            elif is_operator(token):
                print("Is an Operator")
            elif is_parenthesis(token):
                print("Is a Parenthesis")
            else:
                print("Not accepted")

        ast = parse_expr(tokens)
        print("String Accepted" if ast else "String Rejected")
        print("------------------------")
        print("Syntax Analysis : ")
        print("------------------------")
        if ast:
            print("Parsed value:")
            parsed_dict = ast.to_dict()
            print_ast_dict(parsed_dict)

if __name__ == "__main__":
    main()

------------------------
Lexical Analysis : 
------------------------
2 Is a Value
- Is an Operator
2 Is a Value
String Accepted
------------------------
Syntax Analysis : 
------------------------
Parsed value:
type: BinaryExpression
op: -
left:
  type: NumericLiteral
  value: 2
right:
  type: NumericLiteral
  value: 2
